<a href="https://colab.research.google.com/github/otabekds/Web-scrapping-project/blob/main/Kun_uz_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The project is over. The project publishes the latest 8 news on kun.uz in the form of a graph, rating each of the 10 words according to how many times they came.
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import plotly.express as px

def request_get(url):
    return requests.get(url)

def extract(respons):
    res_url, res_name = np.array([]), np.array([])
    html = BeautifulSoup(respons.text, "html.parser")
    soup = html.find("div", class_="mb-25")
    linkes = soup.find_all('a', class_='news-lenta')
    texts = soup.find_all('span', class_='news-lenta__title')
    res_name = [x.text for  x in texts]
    for i, v in enumerate(linkes):
      w =  str(linkes[i]).split()[2][6:-2]

      res_url = np.append(res_url,w)
    return res_name, res_url

def extract2(urls):
  result_respons = []
  for x in urls:
    w = request_get(f"https://kun.uz{x}")
    html  = BeautifulSoup(w.text, "html.parser")
    soup  = html.find_all("div", class_="single-content")
    for x in soup:
      result_respons.append(x.text.strip().split()[:-18])
  return result_respons

def transform(words):
  result_words = [[],[],[],[],[],[],[],[]]

  kic,  kic_idx = ['ni','da','gi','di','ga','lar','dan','gan','gani','ning','lari'],[-2,-2,-2,-2,-2,-3,-3,-3,-4,-4,-4] # I used these words to remove the suffixes at the end and I removed all the suffixes.
  stop_words = np.array(["foto:",'dedi','bermoqda?', 'haqida','har', 'bir','olti','mos','xil','keng','taga', 'bu','bosh','yil',' haqi' ,'nima','uzoq', 'kim', 'va', 'olib', 'o‘tkazish', 'hamda', 'bo‘ladi?', 'kengaymoqda,', 'lekin', 'biroq', 'chunki', 'agar', 'uchun', 'gar', 'balki', 'go’yo', 'basharti', 'yana', 'garchi', 'go’yoki', 'holbuki', 'vaholanki', 'yoki', 'ham', 'ba`zan', 'uchun?', 'anglatadi?', '-', 'yana', 'bilan', 'nega','deb'])
  for idx, x in enumerate(words):
    for i in x:

      for k, v in zip(kic,kic_idx):
        if i[v:] == k:
          i = i[:v]
      if i.lower() in stop_words:
        pass
      elif len(i) < 3:
        pass
      elif i.isalpha():
        result_words[idx].append(i)
  return result_words

def coun_func(words):
  res = [{},{},{},{},{},{},{},{},{}]
  for idx, x in enumerate(words):
    for i in x:
        w = words[idx].count(i)
        if i not in res[idx]:
          res[idx][i] = w 
  return res

def main():
    respons = request_get("https://kun.uz/uz/")
    soup = extract(respons)   # At 1 the link and at 0 the news name returns.
    news_name, id = soup[0], range(1,9)  # it contains the name of all the news.
    news_words = extract2(soup[1])  # In this case, the text of all news returns in a single list.
    ready_words = transform(news_words) # in which I cleared the words
    count_words = coun_func(ready_words) # in which I counted the words and returned them as dict
    for idx, x in enumerate(count_words[:-1]):
      data = {'Soni' : count_words[idx].values(), "So'zlar" : count_words[idx].keys()}
      df = pd.DataFrame(data)
      df = df.sort_values("Soni")
      data = df.tail(10).reset_index()
      data.index.name = 'Miqdor'
      
      fig = px.bar(data, x='Soni', y="So'zlar", title=f"Yangilik nomi:   {news_name[idx]}", color=data.index , orientation='h', width=1200, height=450)
      fig.show()

main()  